# ch2. 마케팅 문구 생성 프롬프트 엔지니어링

이번 챕터에서는 마케팅 문구를 생성하기 위한 프롬프트를 작성해보겠습니다.

## chatGPT API 사용 설정

In [2]:
import os
from openai import OpenAI
from getpass import getpass

In [4]:
openai_api_key = getpass("OPENAI_API_KEY")

OPENAI_API_KEY ········


In [5]:
client = OpenAI(
    api_key=openai_api_key
)

In [6]:
def print_streaming_response(response):
    for chunk in response:
        delta = chunk.choices[0].delta
        if delta.content:
            print(delta.content, end="")

In [7]:
messages = [{"role": "user", "content": "안녕하세요"}]
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    stream=True
)
print_streaming_response(response)

안녕하세요! 어떻게 도와드릴까요?

## 프롬프트 엔지니어링

### AI에게 역할부여

광고 문구를 생성하기 위해서 먼저 system message를 이용해서 AI에게 "전문 카피라이터" 라는 역할을 부여하겠습니다.  
그 다음, 유저로부터 프롬프트만 전달받아서 응답을 리턴하도록 함수화 하겠습니다.

In [8]:
def request_chat_completion(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "당신은 전문 카피라이터입니다."},
            {"role": "user", "content": prompt}
        ],
        stream=True
    )
    return response

"카누" 라는 커피 브랜드의 광고 문구를 작성해달라는 간단한 프롬프트를 넣어서 텍스트를 생성해보겠습니다.

In [9]:
prompt = "'카누'라는 커피믹스 브랜드의 SNS 마케팅을 위한 카피를 3개 생성해줘"
response = request_chat_completion(prompt)
print_streaming_response(response)

물론입니다! '카누' 커피믹스 브랜드를 위한 SNS 마케팅 카피 3개를 아래와 같이 제안드립니다:

---

1. **"한 스푼의 행복, 카누와 함께!"**  
   아침이 다가오면 카누 한 잔으로 시작하세요. 부드러운 맛과 진한 향, 언제 어디서나 손쉽게 즐길 수 있는 카누가 당신의 일상에 작은 행복을 더해드립니다. ☕✨ #카누 #아침의행복

2. **"모든 순간이 특별해지는 커피"**  
   커피 한 잔의 힘, 그 특별한 순간을 카누와 함께하세요. 소중한 사람들과의 만남도, 혼자만의 여유도 카누가 함께합니다. 오늘의 당신을 위해 완벽한 선택! ☕❤️ #카누 #특별한순간

3. **"카누, 나만의 휴식시간"**  
   바쁜 하루 속 잠깐의 휴식, 카누 커피와 함께하세요. 간편하게 즐기는 프리미엄 맛, 언제 어디서나 나만의 시간을 만들어 줍니다. 당신의 소중한 순간을 카누가 더욱 특별하게! ☕🌟 #카누 #휴식의미학

--- 

이러한 카피들은 카누의 브랜드 이미지와 소비자와의 감성적인 연결을 높이는 데 도움이 될 것입니다.

결과를 보면 마케팅 문구를 만들어주긴 하지만 지나치게 길고, 재치가 느껴지지 않습니다.  
프롬프트 엔지니어링을 이용해서 우리가 원하는 대로 텍스트를 생성하도록 제어해보겠습니다.

### 프롬프트 생성

마케팅 문구를 생성하고자 하는 제품명과 제품에 대한 설명을 파라미터로 전달받아서 프롬프트를 생성해보겠습니다.  
이처럼 파라미터를 전달받는 경우, AI에게 먼저 수행해야할 지시를 알려주고 구분자를 사용해서 그 사이에 파라미터를 지정해주는 것이 좋습니다.  
지시 다음 부분에 최대 길이나 어떤 느낌이 나면 좋겠다는 걸 적어주세요.

In [10]:
def generate_prompt(product_name, product_desc):
    prompt = f"""
제품 혹은 브랜드를 SNS에 광고하기 위한 문구를 생성해주세요.
자극적이고 창의적으로 작성하세요.
명사 위주로 간결하게 작성하세요.
---
제품명: {product_name}
제품설명: {product_desc}
---
""".strip()
    return prompt

In [11]:
prompt = generate_prompt(
    product_name="카누",
    product_desc="집에서도 카페 느낌의 아메리카노 맛을 내는 커피 믹스"
)

In [12]:
print(prompt)

제품 혹은 브랜드를 SNS에 광고하기 위한 문구를 생성해주세요.
자극적이고 창의적으로 작성하세요.
명사 위주로 간결하게 작성하세요.
---
제품명: 카누
제품설명: 집에서도 카페 느낌의 아메리카노 맛을 내는 커피 믹스
---


In [14]:
response = request_chat_completion(prompt)
print_streaming_response(response)

“카누: 집에서 즐기는 카페의 한 잔.  
커피의 향, 진한 맛, 완벽한 순간.  
당신의 일상에 한 방울의 카페를!”  
#카누 #홈카페 #아메리카노 #맛의여행

### 생성하는 텍스트 길이, 개수 조정

이번에는 생성하는 텍스트의 길이와 개수를 프롬프트로 조정해보겠습니다.  
generate_prompt 함수에 파라미터를 추가해보겠습니다.  
개수의 경우 문구를 n개 생성해달라고 적어주면 잘 반영됩니다.  
텍스트 길이 조정의 경우엔 몇 자 이내로 적어달라는 것 보다는, 몇 단어 이내로 적어달라는 것이 좋으며, 앞에 "반드시"를 붙여주세요.

In [15]:
def generate_prompt(product_name, product_desc, num, max_length):
    prompt = f"""
제품 혹은 브랜드를 SNS에 광고하기 위한 문구를 {num}개 생성해주세요.
자극적이고 창의적으로 작성하세요.
명사 위주로 간결하게 작성하세요.
반드시 {max_length} 단어 이내로 작성해주세요.
---
제품명: {product_name}
제품설명: {product_desc}
---
""".strip()
    return prompt

In [16]:
prompt = generate_prompt(
    product_name="카누",
    product_desc="집에서도 카페 느낌의 아메리카노 맛을 내는 커피 믹스",
    num=5,
    max_length=7
)

In [17]:
print(prompt)

제품 혹은 브랜드를 SNS에 광고하기 위한 문구를 5개 생성해주세요.
자극적이고 창의적으로 작성하세요.
명사 위주로 간결하게 작성하세요.
반드시 7 단어 이내로 작성해주세요.
---
제품명: 카누
제품설명: 집에서도 카페 느낌의 아메리카노 맛을 내는 커피 믹스
---


In [18]:
response = request_chat_completion(prompt)
print_streaming_response(response)

1. 카페 분위기, 집에서 즐기세요!  
2. 카누, 한 모금의 행복한 순간!  
3. 아메리카노, 당신의 집에서 재탄생!  
4. 카누로 완성하는 특별한 커피 시간!  
5. 집에서 카페, 카누와 함께!  

### 키워드 포함

마케팅 문구에 반드시 포함했으면 하는 키워드가 있다면 프롬프트에 포함할 수 있습니다.  
generate_prompt에 파라미터를 추가하고, 프롬프트를 약간 수정해보겠습니다.

In [19]:
def generate_prompt(product_name, product_desc, num, max_length, keywords):
    prompt = f"""
제품 혹은 브랜드를 SNS에 광고하기 위한 문구를 {num}개 생성해주세요.
자극적이고 창의적으로 작성하세요.
명사 위주로 간결하게 작성하세요.
반드시 {max_length} 단어 이내로 작성해주세요.
키워드가 주어질 경우, 반드시 키워드 중 하나를 포함해야 합니다.
---
제품명: {product_name}
제품설명: {product_desc}
키워드: {keywords}
---
""".strip()
    return prompt

In [20]:
prompt = generate_prompt(
    product_name="카누",
    product_desc="집에서도 카페 느낌의 아메리카노 맛을 내는 커피 믹스",
    num=5,
    max_length=7,
    keywords=["브라질", "풍미", "향기"]
)

In [21]:
response = request_chat_completion(prompt)
print_streaming_response(response)

1. 브라질의 풍미, 집에서 느껴보세요!  
2. 향기로운 카페, 카누와 함께 하세요!  
3. 집에서 브라질의 커피 향기 만끽!  
4. 카누, 풍미가 가득한 커피 마법!  
5. 향기와 풍미, 카누의 특별한 조화!

## 정리

지금까지 마케팅 문구를 생성하기 위한 프롬프트 엔지니어링을 배워봤습니다.  
어떤가요? 굉장히 쉽지 않나요? 내가 결과로 얻고 싶은 텍스트를 자연어로 묘사하고, 파라미터만 쏙쏙 넣어주면 됩니다.  
이제 streamlit을 이용해서 UI를 개발하고, 여기에 마케팅 문구 생성 기능을 적용하겠습니다.